# Imports

In [2]:
import csv
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
target = 'enc_group'

In [3]:
def down_sampling_preprocessing(df):
  df['unique_id'] = df.reset_index().index
  df = df.loc[:,df.isnull().sum()/len(df) <0.3]

  df = df[(df["gname"] != 'Unknown')]


  # list1 = ["Irish Republican Army (IRA)"]
  # df = df[~df.gname.isin(list1)]
  df['eventid'] = df['eventid'].astype('int64')
  categorical = ['extended','crit1','crit2','crit3','doubtterr','alternative','alternative_txt','multiple','enc_country','enc_province',\
                 'country_txt','enc_region','region_txt','vicinity','specificity','enc_attacktype','attacktype','enc_weapon_type',               'weapon_type','enc_weapon_subtype','weapon_subtype','success','suicide','target_entity','enc_target','target_type',
                 'target_subtype','enc_nationality','nationality','claimed','property','cross_border','ideological_international',\
                 'province','entity','group','hostages/kidnapping','city','INT_MISC','INT_ANY','individual']
  numerical = ['eventid','year','month','day','latitude','longitude','killed','nkillter','wounded','province']
  text = ['approxdate','summary','related','location','motive']
  numerical = ['eventid','year','month','day','latitude','longitude','killed','nkillter','wounded']
  text = ['approxdate','summary','related','location','motive']
  df.rename(columns = {'iyear':'year','imonth':'month','iday':'day','region':'enc_region', \
                       'attacktype1_txt':'attacktype','attacktype1':'enc_attacktype',\
                       'provstate':'enc_province','country':'enc_country',\
                       'targtype1_txt':'target_type','targtype1':'enc_target',\
                       'targsubtype1_txt':'target_subtype', \
                       'target1':'target_entity','weaptype1':'enc_weapon_type', \
                       'weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype','weapsubtype1':'enc_weapon_subtype',\
                       'corp1':'entity','natlty1':'enc_nationality','natlty1_txt':'nationality',\
                       'gname':'enc_group','nkill':'killed','nwound':'wounded',\
                       'targsubtype':'enc_target_subtype',\
                       'ishostkid':'hostages/kidnapping','INT_LOG':'cross_border',\
                       'INT_IDEO':'ideological_international'}, inplace = True)
  for item in categorical:
      if item not in df.columns:
          pass
      else:
          df[item] = df[item].astype('category')
  df["enc_city"] = df["city"].cat.codes
  df["enc_city"] = df["enc_city"].astype('category')
  df["enc_city"] = df["city"].cat.codes
  df["enc_city"] = df["enc_city"].astype('category')
  df["enc_province"] = df["enc_province"].cat.codes
  df["enc_province"] = df["enc_province"].astype('category')
  df["country_txt"] = df["country_txt"].cat.codes
  df["region_txt"] = df["region_txt"].cat.codes
  df["enc_attacktype"] = df["attacktype"].cat.codes
  df["enc_target_type"] = df["target_type"].cat.codes
  df["enc_target_subtype"] = df["target_subtype"].cat.codes
  df["enc_target_entity"] = df["target_entity"].cat.codes
  df["enc_nationality"] = df["nationality"].cat.codes
  df["enc_weapon_type"] = df["weapon_type"].cat.codes
  df['attack_date'] = pd.to_datetime(df[['year','month','day']], errors = 'coerce')
  df = df.drop(columns = ['targsubtype1','INT_ANY','year','dbsource','month','day','crit1','crit2','crit3',"city",'country_txt','region_txt','attacktype','target_type','target_subtype','target_entity','nationality','weapon_type'])

  if "weapdetail" in df.columns:
    df = df.drop(columns = ['weapdetail'])

  if "summary" in df.columns:
    df = df.drop(columns = ['summary'])

  if "scite1" in df.columns:
    df = df.drop(columns = ['scite1'])


  if "weapon_subtype" in df.columns:
    df = df.drop(columns = ['weapon_subtype'])

  return df

# Mount google drive

In [4]:
# Uncomment below two lines if data is placed on Google Drive or need to connect drive
#from google.colab import drive
#drive.mount('/content/drive')

# Down sampling preprocessing

# Get Stats

In [5]:
import pandas as pd
import numpy as np

def get_stats(df1):
    # Calculate the number of attacks per group
    group_attacks = df1['enc_group'].value_counts().reset_index()
    group_attacks = group_attacks.sort_values(by='count', ascending=False)
    # group_attacks = group_attacks.sort_values(by='count', ascending=False)
    group_attacks.columns = ['gname', 'group_count']
    group_attacks = group_attacks.head(20)
    # group_attacks = group_attacks[group_attacks['group_count'] > 150]


    # Assuming group_attacks is your DataFrame

    # Calculate the sum of the counts
    total_count = group_attacks['group_count'].sum()

    # Calculate the fraction per 1000 rows
    frac_per_1000 = round(1000 / total_count)

    # Calculate the exact number of rows each group should contribute
    group_attacks['fraction_per_1000'] = (group_attacks['group_count'] * frac_per_1000).round().astype(int)
    de = group_attacks
    # Calculate the total number of rows after rounding
    total_rows = group_attacks['fraction_per_1000'].sum()

    # If the total number of rows is less than 1000, distribute the remaining rows among groups
    if total_rows < 1000:
        remaining_rows = 1000 - total_rows
        group_attacks['fraction_per_1000'] += np.floor(remaining_rows * (group_attacks['group_count'] / total_count))

    # If the total number of rows is greater than 1000, reduce the number of rows in some groups
    elif total_rows > 1000:
        excess_rows = total_rows - 1000
        group_attacks['fraction_per_1000'] -= np.ceil(excess_rows * (group_attacks['group_count'] / total_count))

    import pandas as pd

    downsampled_data = pd.DataFrame()

    # Iterate through each group in the top 20 groups
    for index, row in group_attacks.head(20).iterrows():
        # Extract data for the current group
        group_data = df1[df1['enc_group'] == row['gname']]

        # Calculate the number of samples to take for the current group
        num_samples = int(row["fraction_per_1000"])

        # Sample rows without replacement based on the calculated proportion

        sampled_data = group_data.sample(n=num_samples, replace=False, random_state=42)

        # Append the downsampled data to the overall downsampled_data DataFrame
        # downsampled_data = downsampled_data.append(sampled_data)
        downsampled_data = pd.concat([downsampled_data, sampled_data], ignore_index=True)

    return downsampled_data

#Encoding preprocessing

In [6]:
import pandas as pd

def encode_preprocessing(df):
    """
    Preprocess the input DataFrame by applying various data cleaning and transformation steps.

    Parameters:
        df (DataFrame): Input DataFrame to be preprocessed.

    Returns:
        DataFrame: Preprocessed DataFrame.
    """
    # Filter out rows where enc_group is 'Unknown'
    # df = df[(df["enc_group"] != 'Unknown')]
    # df['unique_id'] = df.reset_index().index
    # Convert 'eventid' column to int64
    df['eventid'] = df['eventid'].astype('int64')
    categorical = ['extended','crit1','crit2','crit3','doubtterr','alternative','alternative_txt','multiple','enc_country','enc_province',\
                 'country_txt','enc_region','region_txt','vicinity','specificity','enc_attacktype','attacktype','enc_weapon_type',               'weapon_type','enc_weapon_subtype','weapon_subtype','success','suicide','target_entity','enc_target','target_type',
                 'target_subtype','enc_nationality','nationality','claimed','property','cross_border','ideological_international',\
                 'province','entity','group','hostages/kidnapping','city','INT_MISC','INT_ANY','individual']
    numerical = ['eventid','year','month','day','latitude','longitude','killed','nkillter','wounded','province']
    text = ['approxdate','summary','related','location','motive']
    numerical = ['eventid','year','month','day','latitude','longitude','killed','nkillter','wounded']
    text = ['approxdate','summary','related','location','motive']
    df.rename(columns = {'iyear':'year','imonth':'month','iday':'day','region':'enc_region', \
                         'attacktype1_txt':'attacktype','attacktype1':'enc_attacktype',\
                         'provstate':'enc_province','country':'enc_country',\
                         'targtype1_txt':'target_type','targtype1':'enc_target',\
                         'targsubtype1_txt':'target_subtype', \
                         'target1':'target_entity','weaptype1':'enc_weapon_type', \
                         'weaptype1_txt':'weapon_type','weapsubtype1_txt':'weapon_subtype','weapsubtype1':'enc_weapon_subtype',\
                         'corp1':'entity','natlty1':'enc_nationality','natlty1_txt':'nationality',\
                         'gname':'enc_group','nkill':'killed','nwound':'wounded',\
                         'targsubtype':'enc_target_subtype',\
                         'ishostkid':'hostages/kidnapping','INT_LOG':'cross_border',\
                         'INT_IDEO':'ideological_international'}, inplace = True)
    for item in categorical:
        if item not in df.columns:
            pass
        else:
            df[item] = df[item].astype('category')
    df["enc_city"] = df["city"].cat.codes
    df["enc_city"] = df["enc_city"].astype('category')
    df["enc_city"] = df["city"].cat.codes
    df["enc_city"] = df["enc_city"].astype('category')
    df["enc_province"] = df["enc_province"].cat.codes
    df["enc_province"] = df["enc_province"].astype('category')
    df["country_txt"] = df["country_txt"].cat.codes
    df["region_txt"] = df["region_txt"].cat.codes
    df["enc_attacktype"] = df["attacktype"].cat.codes
    df["enc_target_type"] = df["target_type"].cat.codes
    df["enc_target_subtype"] = df["target_subtype"].cat.codes
    df["enc_target_entity"] = df["target_entity"].cat.codes
    df["enc_nationality"] = df["nationality"].cat.codes
    df["enc_weapon_type"] = df["weapon_type"].cat.codes
    df['attack_date'] = pd.to_datetime(df[['year','month','day']], errors = 'coerce')
    df = df.drop(columns = ['targsubtype1','INT_ANY','year','dbsource','month','day','crit1','crit2','crit3',"city",'country_txt','region_txt','attacktype','target_type','target_subtype','target_entity','nationality','weapon_type'])

    if "weapdetail" in df.columns:
      df = df.drop(columns = ['weapdetail'])

    if "summary" in df.columns:
      df = df.drop(columns = ['summary'])

    if "scite1" in df.columns:
      df = df.drop(columns = ['scite1'])


    if "weapon_subtype" in df.columns:
      df = df.drop(columns = ['weapon_subtype'])
     # Drop columns specified in 'text' list
    text = ['targsubtype1', 'latitude', 'longitude', 'region_txt', 'corp1', 'weapsubtype1',
             'country_txt', 'dbsource', 'summary', 'crit1', 'crit2', 'crit3', 'targtype1_txt',
             'targsubtype1_txt', 'attacktype1_txt', 'natlty1_txt', 'weaptype1_txt', 'provstate',
             'weapsubtype1_txt', 'scite1', 'scite2','scite3','related','addnotes','hostkidoutcome_txt','propcomment','resolution']
    for i in text:
        if i in df.columns:
            df = df.drop(columns=[i])

    # Convert 'enc_target' column to category and encode it
    df["enc_target"] = df["enc_target"].astype('category')
    df["enc_target"] = df["enc_target"].cat.codes

    # Convert 'enc_city' column to category and encode it
    df["enc_city"] = df["enc_city"].astype('category')
    df["enc_city"] = df["enc_city"].cat.codes

    # Fill missing values with 0 for specified columns
    replac_na_values = ['nkill', 'nkillus', 'nkillter', 'nwound', 'nwoundus', 'nwoundte']
    for i in replac_na_values:
        if i in df.columns:
            df[i] = df[i].fillna(0)

    # Rename 'city' column to 'enc_city'
    df.rename(columns={'city': 'enc_city'}, inplace=True)
    df["enc_city"] = df["enc_city"].astype('category')
    df["enc_city"] = df["enc_city"].cat.codes


    # Convert all columns to categorical and then one-hot encode them
    for item in df:
        df[item] = df[item].astype('category')

    unique_id = df[["unique_id"]]

    df["enc_group"] = df["enc_group"].str.replace(' ', '_')
    enc_group = df[["enc_group"]]

    df = df.drop(columns=['eventid','unique_id','enc_group'])
    df = pd.get_dummies(df)

    # df.insert(0,"unique_id",unique_id)
    df.insert(1,"enc_group",enc_group)
    df.reset_index(inplace=True)
    # print(type(df))
    # print(df.columns)

    df.columns.values[0] = "unique_id"
    # print(df.head(5))
    # Drop 'enc_group' and 'eventid' columns
    # df = df.drop(columns=['enc_group', 'eventid'])

    return df

# Split and Save data to CSV

---



In [7]:
import os
import pandas as pd

def split_and_save_original_data(df, downsampled_data, partition):
    # Calculate the number of rows for each set
    print("Number of samples after pre-processing/down sampling",downsampled_data.shape[0])
    print("Number of samples after pre-processing/down sampling",downsampled_data.shape[1])
    _ids = downsampled_data.unique_id.to_list()
    _data = df[df['unique_id'].isin(_ids)]
    encoded_downsampled_data = encode_preprocessing(_data)
    encoded_directory = f"../test/{partition}/"
    if not os.path.exists(encoded_directory):
        os.makedirs(encoded_directory)
    encoded_downsampled_data.to_csv(os.path.join(encoded_directory, "encoded_"+partition + ".csv"),index=False, encoding="ISO-8859-1")
    print(" ")


    total_rows = downsampled_data.shape[0]
    #down_directory = f"/content/drive/MyDrive/GTD/original/{partition}/"+"down_sampled"+"/"
    down_directory = f"../test/{partition}/"+"down_sampled"+"/"

    # Create the directory if it doesn't exist
    if not os.path.exists(down_directory):
        os.makedirs(down_directory)

    # Save the train DataFrame to CSV
    downsampled_data.to_csv(os.path.join(down_directory,"new_downsampled_"+ partition + ".csv"),index=False, encoding="ISO-8859-1")
    train_size = int(total_rows * 0.7)
    val_size = int(total_rows * 0.1)

    # Splitting the data into train, validation, and test sets
    train_data = downsampled_data.iloc[:train_size]
    val_data = downsampled_data.iloc[train_size:train_size + val_size]
    test_data = downsampled_data.iloc[train_size + val_size:]
    print("Number of Features after preprocessing and before encoding")
    save_data(train_data, val_data,test_data,partition)
    print(" ")

    # Splitting the data into train, validation, and test sets
    encoded_train_data = encoded_downsampled_data.iloc[:train_size]
    encoded_val_data = encoded_downsampled_data.iloc[train_size:train_size + val_size]
    encoded_test_data = encoded_downsampled_data.iloc[train_size + val_size:]
    print("Enccoded Features",encoded_downsampled_data.shape)
    print(" ")
    print("Number of one hot encoded Features after preprocessing")
    save_data(encoded_train_data, encoded_val_data,encoded_test_data,partition,"encoded")
    print(" ")


def save_data(train_data, val_data ,test_data,partition,folder="down_sampled"):


    # train_ids = train_data.unique_id.to_list()
    # val_ids = val_data.unique_id.to_list()
    # test_ids = test_data.unique_id.to_list()

    # train_data = df[df['unique_id'].isin(train_ids)]

    # Define the train directory path
    #train_directory = f"/content/drive/MyDrive/GTD/original/{partition}/"+folder+"/train/"
    train_directory = f"../test/{partition}/"+folder+"/train/"

    # Create the directory if it doesn't exist
    if not os.path.exists(train_directory):
        os.makedirs(train_directory)

    # Save the train DataFrame to CSV
    train_data.to_csv(os.path.join(train_directory, partition + ".csv"),index=False, encoding="ISO-8859-1")

    # val_data = df[df['unique_id'].isin(val_ids)]
    # Define the validation directory path
    #val_directory = f"/content/drive/MyDrive/GTD/original/{partition}/"+folder+"/val/"
    val_directory = f"../test/{partition}/"+folder+"/val/"

    # Create the directory if it doesn't exist
    if not os.path.exists(val_directory):
        os.makedirs(val_directory)

    # Save the validation DataFrame to CSV
    val_data.to_csv(os.path.join(val_directory, partition + ".csv"),index=False, encoding="ISO-8859-1")

    # Define the test directory path
    #test_directory = f"/content/drive/MyDrive/GTD/original/{partition}/"+folder+"/test/"
    test_directory = f"../test/{partition}/"+folder+"/test/"

    # test_data = df[df['unique_id'].isin(test_ids)]
    # Create the directory if it doesn't exist
    if not os.path.exists(test_directory):
        os.makedirs(test_directory)

    # Save the test DataFrame to CSV
    test_data.to_csv(os.path.join(test_directory, partition + ".csv"),index=False, encoding="ISO-8859-1")

    # Print the shapes of the resulting sets
    # print("Train set shape:", train_data.shape)
    # print("Validation set shape:", val_data.shape)
    # print("Test set shape:", test_data.shape)



    print("Data Preparation, train size",train_data.shape)
    print("Data Preparation, val size",val_data.shape)
    print("Data Preparation, test size",test_data.shape)


# split_and_save_data(downsampled_data, "your_partition_name")


# Iterator

In [8]:

# List of partitions
partitions = ["data_1970_80","data_1981_95", "data_96_2010", "data_2011_12", "data_2013_14","data_2015_17"]

for partition in partitions:
    # Read the data
    print("------------------- "+partition+" -------------------")
    #df = pd.read_csv(f"/content/drive/MyDrive/GTD/original/{partition}/{partition}.csv", encoding="ISO-8859-1", low_memory=False)
    df = pd.read_csv(f"../original/{partition}/{partition}.csv", encoding="ISO-8859-1", low_memory=False)
    print("Number of samples before down sampling",df.shape[0])
    print("Number of Features after down sampling",df.shape[1])
    df1 = down_sampling_preprocessing(df)
    print(df1.shape)
    downsampled_data = get_stats(df1)
    print(downsampled_data.shape)

    split_and_save_original_data(df, downsampled_data, partition)

    print("----------------------------------------------------")
    print(" ")



------------------- data_1970_80 -------------------
Number of samples before down sampling 12576
Number of Features after down sampling 135
(8597, 35)
(988, 35)
Number of samples after pre-processing/down sampling 988
Number of samples after pre-processing/down sampling 35
 
Number of Features after preprocessing and before encoding
Data Preparation, train size (691, 35)
Data Preparation, val size (98, 35)
Data Preparation, test size (199, 35)
 
Enccoded Features (988, 3044)
 
Number of one hot encoded Features after preprocessing
Data Preparation, train size (691, 3044)
Data Preparation, val size (98, 3044)
Data Preparation, test size (199, 3044)
 
----------------------------------------------------
 
------------------- data_1981_95 -------------------
Number of samples before down sampling 48676
Number of Features after down sampling 135
(32925, 35)
(991, 35)
Number of samples after pre-processing/down sampling 991
Number of samples after pre-processing/down sampling 35
 
Number o